<h1> Classificadores Avançados </h1>

<h2> 1. Árvores de Decisão <h2>

In [3]:
%matplotlib inline

In [9]:
import matplotlib.pyplot as plt
import numpy as np 
import math

<h2> Primeiro Vamos Criar a Estrutura de Dados </h2>

In [40]:
class Noh:
    def __init__(self,rot,valores=()): # rotulo do no + lista de atributos
        self.rotulo =  rot
        self.valores = valores
        self.filhos  = [None for i in valores]
    
    def addFilho(self,valor,noh):
        pos = self.valores.index(valor)
        self.filhos[pos]=noh
    
    def __repr__(self):
        st = "< {} : {} : {} > \n".format(self.rotulo,self.valores,self.filhos)
        return st

In [41]:
#
#
class ArvoreDecisao:
    # construtor da classe
    def __init__(self,noh):
        self.raiz = noh
    
    # adiciona um nó filho a um dado pai
    def addNoh(self, p, valor, noh):
        #localiza o no pai
        pais = self.locNoh(self.raiz,p)
        for pai in pais:
            pai.addFilho(valor,noh)
        
    #localiza o noh - Usa Busca em Largura
    def locNoh(self,raiz,rotulo):
        lista=[raiz]
        res = []
        while len(lista) > 0:
            atual = lista.pop(0)
            if atual == None: continue
            if atual.rotulo == rotulo:
                res.append(atual)
            else:
                for v in atual.filhos:
                    lista.append(v)
        return res
    
    #imprime a arvore
    def imprimir(self,raiz,tab=""):
        if raiz == None: 
            print(tab,"X")
        else:
            print(tab,raiz.rotulo)
        tab+="\t | "
        for i,j in zip(raiz.valores,raiz.filhos):
            print(tab,"[{}]".format(i))
            if j != None: self.imprimir(j,tab)
    
    # faz inferencia, ou seja encontra o ramos e a saida(folha)
    def inferencia(self, nr):
        if nr.valores == "{}":
            return nr.rotulo 
        resposta = input("Qual o valor de {} ( {} )".format(nr.rotulo,nr.valores)).upper()
        pos = nr.valores.index(resposta)
        return self.inferencia(nr.filhos[pos])
        
    

</h2> Calculos ...</h3>

In [33]:
#vamos definir nossa função entropia
def entropia(a,b): # a = quantidade de elementos a e b quant de elementos b
    if a == 0 or b == 0: return 0
    total = a+b
    pa = a/total
    pb = b/total
    return -pa*math.log(pa,2) - pb*math.log(pb,2) 


In [34]:
entropia(0,4)

0

In [35]:
def r(lista): # lista de tuplas dos filhos (Positivos,Negativos)
    P = 0
    N = 0
    R = 0 
    for i in lista:
        P += i[0]
        N += i[1]
    for f in lista:
        R += (f[0]+f[1])/(P+N)*entropia(f[0]/(f[0]+f[1]),f[1]/(f[0]+f[1]))
    return R
    

In [36]:
r([(1,2),(1,2),(1,2),(1,2)])

0.9182958340544896

In [38]:
r([(3,2),(4,0),(3,2)])

0.6935361388961918

In [42]:
datasetRestaurante = [["CLIENTELA","TIPO","FICOU"],
                       ["LOTADO","LACARTE","N"],
                       ["MEDIO","LACARTE","S"],
                       ["VAZIO","LACARTE","N"],
                       ["LOTADO","SELF","N"],
                       ["MEDIO","SELF","S"],
                       ["VAZIO","SELF","N"],  
                       ["LOTADO","FAST","N"],
                       ["MEDIO","FAST","S"],
                       ["VAZIO","FAST","N"],
                       ["LOTADO","SUSHI","N"],
                       ["MEDIO","SUSHI","S"],
                       ["VAZIO","SUSHI","N"],
                      ]

In [43]:
datasetTempo = [["Tempo", "Temperatura", "Humidade", "Ventando", "JogarTennis"],
                ["Ensolarado", "Quente", "Alta", "Não", "N"],
                ["Ensolarado", "Quente", "Alta", "Sim", "N"],
                ["Nublado", "Quente", "Alta", "Não", "S"],
                ["Chuva", "Média", "Alta", "Não", "S"],
                ["Chuva", "Fria", "Normal", "Não", "S"],
                ["Chuva", "Fria", "Normal", "Sim", "N"],
                ["Nublado", "Fria", "Normal", "Sim", "S"],
                ["Ensolarado", "Média", "Alta", "Não", "S"],
                ["Ensolarado", "Fria", "Normal", "Não", "S"],
                ["Chuva", "Média", "Normal", "Não", "S"],
                ["Ensolarado", "Média", "Normal", "Sim", "S"],
                ["Nublado", "Média", "Alta", "Sim", "S"],
                ["Nublado", "Quente", "Normal", "Não", "S"],
                ["Chuva", "Média", "Alta", "Sim", "N"],]

In [45]:
# conta os positivo e nagativos. ignora a linha de rotulos
def totalPN(ds):
    lista = [i[-1] for i in ds]
    p = lista.count("S")
    n = lista.count("N")
    return p,n

In [46]:
# poda o dataset
def subDs(ds,at,val):
    pos = ds[0].index(at)
    sub = [ds[0]] + [i for i in ds if i[pos] == val ]
    return sub

In [47]:
#
def removeColuna(ds,atributo):
    sub = [i[:] for i in ds ] # copia profunda
    pos = ds[0].index(atributo)
    for i in sub: del i[pos]
    return sub

In [48]:
def listaValores(ds):
    valores = dict()
    feats = ds[0][:-1]
    feats = ds[0]
    #print(feats)
    for f in feats:
        pos = feats.index(f)
        listVal = set([v[pos] for v in ds[1:] ])
        #print(listVal)
        valores[f] = listVal
    return valores    

In [49]:
def calculaGanhoInformacao(dados):
    p,n = totalPN(dados)
    entropia_inicial = entropia(p,n)
    listaFeat = dados[0][:-1]
    listval = listaValores(dados)
    entResto = 0
    ganhos = dict()
    for k,v in listval.items():
        for i in v:
            ss = subDs(dados,k,i)
            sp,sn = totalPN(ss)
            sentropia = entropia(sp,sn)
            entResto += (sp+sn)/(p+n) * sentropia
        gan = entropia_inicial - entResto
        ganhos[k] = gan
    return sorted(ganhos.items(), key=lambda x: x[1],reverse=True)

In [50]:
def selecionaAtributoValor(ds):
    if len(ds[0]) == 1:
        atributo = "".join(ds[0])
        v = []
        for i in ds[1:]:
            if i not in v: v.append("".join(i))
    elif len(ds[0]) == 2:
        atributo = ds[0][0]
        v = set([i[0] for i in ds[1:]])
    else:
        atributo = calculaGanhoInformacao(ds)[0][0]
        vals = listaValores(ds)
        v = vals.get(atributo)
    return atributo,tuple(v)

In [51]:
def geraArvoreID3(ds,noh=None,arco=None):
    if not ds:return
    if noh == None: return
    a,v = selecionaAtributoValor(ds)
    n = Noh(a,tuple(v))
    noh.addFilho(arco,n)
    for i in n.valores:
        nds  = removeColuna(subDs(ds,a,i),a)
        if not nds[-1]:break
        geraArvoreID3(nds,n,i)

def geraArvore(ds):
    #print("Criado a raiz")
    if len(ds[0])<3:
        return "Muito pequeno dataset"
    a,v = selecionaAtributoValor(ds)
    n = Noh(a,v)
    dt = ArvoreDecisao(n)
    #
    # recursao
    #
    for i in n.valores:
        nds  = removeColuna(subDs(ds,a,i),a)
        if not nds[0]:break
        geraArvoreID3(nds,n,i)
    return dt

In [52]:
ad = geraArvore(datasetRestaurante)
ad.imprimir(ad.raiz)

 CLIENTELA
	 |  [MEDIO]
	 |  TIPO
	 | 	 |  [LACARTE]
	 | 	 |  FICOU
	 | 	 | 	 |  [S]
	 | 	 |  [SUSHI]
	 | 	 |  FICOU
	 | 	 | 	 |  [S]
	 | 	 |  [FAST]
	 | 	 |  FICOU
	 | 	 | 	 |  [S]
	 | 	 |  [SELF]
	 | 	 |  FICOU
	 | 	 | 	 |  [S]
	 |  [LOTADO]
	 |  TIPO
	 | 	 |  [LACARTE]
	 | 	 |  FICOU
	 | 	 | 	 |  [N]
	 | 	 |  [SUSHI]
	 | 	 |  FICOU
	 | 	 | 	 |  [N]
	 | 	 |  [FAST]
	 | 	 |  FICOU
	 | 	 | 	 |  [N]
	 | 	 |  [SELF]
	 | 	 |  FICOU
	 | 	 | 	 |  [N]
	 |  [VAZIO]
	 |  TIPO
	 | 	 |  [LACARTE]
	 | 	 |  FICOU
	 | 	 | 	 |  [N]
	 | 	 |  [SUSHI]
	 | 	 |  FICOU
	 | 	 | 	 |  [N]
	 | 	 |  [FAST]
	 | 	 |  FICOU
	 | 	 | 	 |  [N]
	 | 	 |  [SELF]
	 | 	 |  FICOU
	 | 	 | 	 |  [N]


In [53]:
ad2 = geraArvore(datasetTempo)
ad2.imprimir(ad2.raiz)

 Tempo
	 |  [Ensolarado]
	 |  Temperatura
	 | 	 |  [Quente]
	 | 	 |  Humidade
	 | 	 | 	 |  [Alta]
	 | 	 | 	 |  Ventando
	 | 	 | 	 | 	 |  [Não]
	 | 	 | 	 | 	 |  JogarTennis
	 | 	 | 	 | 	 | 	 |  [N]
	 | 	 | 	 | 	 |  [Sim]
	 | 	 | 	 | 	 |  JogarTennis
	 | 	 | 	 | 	 | 	 |  [N]
	 | 	 |  [Fria]
	 | 	 |  Humidade
	 | 	 | 	 |  [Normal]
	 | 	 | 	 |  Ventando
	 | 	 | 	 | 	 |  [Não]
	 | 	 | 	 | 	 |  JogarTennis
	 | 	 | 	 | 	 | 	 |  [S]
	 | 	 |  [Média]
	 | 	 |  Humidade
	 | 	 | 	 |  [Alta]
	 | 	 | 	 |  Ventando
	 | 	 | 	 | 	 |  [Não]
	 | 	 | 	 | 	 |  JogarTennis
	 | 	 | 	 | 	 | 	 |  [S]
	 | 	 | 	 |  [Normal]
	 | 	 | 	 |  Ventando
	 | 	 | 	 | 	 |  [Sim]
	 | 	 | 	 | 	 |  JogarTennis
	 | 	 | 	 | 	 | 	 |  [S]
	 |  [Chuva]
	 |  Temperatura
	 | 	 |  [Fria]
	 | 	 |  Humidade
	 | 	 | 	 |  [Normal]
	 | 	 | 	 |  Ventando
	 | 	 | 	 | 	 |  [Não]
	 | 	 | 	 | 	 |  JogarTennis
	 | 	 | 	 | 	 | 	 |  [S]
	 | 	 | 	 | 	 |  [Sim]
	 | 	 | 	 | 	 |  JogarTennis
	 | 	 | 	 | 	 | 	 |  [N]
	 | 	 |  [Média]
	 | 	 |  Humidade

<h1> Enjoy </h1>